# Привет!
Мы рады что ты смотришь наше тестовое - это круто! Еще круче если ты успешно с ним справишься и тем самым покажешь что ты как минимум сильный джун (или выше).
Для тебя мы приготовили «боевую» задачу практически из нашей работы. По факту мы убрали все ETL и собственно кучу данных оставив только готовые агрегаты и понятный бизнес вэлью.
Описание задачи несколько сложновато, мы это понимаем, тем не менее это также всегда часть работы понять в чем проблема заказчика и что именно стоит сделать

# Задача

Улучшить функцию построения payment profile (сейчас это по сути аналитическая функция model_function и подбор коэффициентов get_coefs) таким образом чтобы минимизировать ошибку описанную ниже.

Решение направить на почту analytics@appquantum.com и соответственно указать контакты для связи.

Что мы ожидаем - что вы либо апгрейдите текущую функцию, либо напишите несколько функций и их заблендите, либо примените какой-то ML алгоритм. Что вам стоит учесть - просто в лоб применить условный catboost не подойдет, т.к. на выходе мы должны получать прогнозируемую линейку кумулятивного АРПУ, а не просто предикт n-ого дня. 

Тут нет единственно верной реализации, понятно что всегда trade off времени и результата, поэтому мы в итоге посмотрим подход, качество кода и в целом работоспособность (текущая функция которую стоит апгрейдить норм работает)



# Описание

Мы строим payment profile накопления ARPU
затем мы этот payment profile применяем к фактическим данным когорт и 
таким образом получаем прогноз АРПУ на 90-й день 
Т.е. если когорта прожила 5-ть дней то к ней применяем коэффииент 5-ого дня

Дальше встает вопрос обучения, оверфитинга и прочего

Мы можем обучиться на 90 днях и иметь полную апроксимированную линейку (пример 1 ниже)
Здесь какая проблема - мы используем только достаточно старых полььзователей которые проджили больше 90 дней, 
а новых не используем

Мы можем обучиться на 30 днях или меньше и использовать апроксимированную линейку на все 90 дней (пример 2 ниже)
Здесь в чем плюс - мы используем достаточно новых пользователей, иногда новые источники заливки 
по которым может не быть больших данных за 90 дней
Здесь какая проблема - мы апроксимировали достаточно хорошо накопление арпу в первые 30 дней 
но можем ошибиться в днях после, что наглядно показано в примере 2, т.е. неправильно оценивается именно «хвост распределения». Динамика первых 30 правильная (т.к. она и апроксимируется) но выручка за 30-м днем оценивается не верно, условно модельное значение 35% выручки после 30 дня а по факту 42%

Итого в чем задача - предложить способ использовать последние 14/30 дней для построения payment profile 
но при этом попытаться избежать проблемы примера 2 когда некорректно оценивается хвост платежей
Т.е. при этом мы предполагаем что вы используете и условно старые данные и маскимально актуальные

здесь имеется ввиду что обновляем линейки мы по последним 14 дням когда например могут добавиться новые источники или выйти апдейт в игре, а соответственно использовать можно любые данные как раз таки здесь мы показываем что линейка за все 90 дней неплохо апроксимируется

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import math
from scipy.optimize import minimize

In [2]:
# задаем стандартную аналитическую экспоненциальную функцию описывающую приращение метрики АРПУ
# ниже будем подбирать к ней коэффициенты

def model_function(c, a, b, x):
    return c * np.exp(-a * x**b)

In [3]:
# функция подбора коэффициентов
# тут что важно - R2 премирует за приближение больших значений, и это в целом нам подходит, 
# т.к. правильно оценить АРПУ в первые дни с одной стороны сложнее т.к. данных мало, 
# с другой стороны важнее, т.к. маркетинг получает результаты в течение пары дней а не ждет недели

def get_coefs(t):
    base_value = t[0]
    
    def ltv_error(x):
        error = 0.0
        for i in range(0, len(t)):
            error += (model_function(t[0], x[0], x[1], i) - t[i])**2
        return math.sqrt(error / len(t))

    x0 = np.array([0.5, 0.5])
    res = minimize(
            ltv_error,
            x0,
            method='nelder-mead',
            options={'xatol': 1e-8, 'disp': True}
    )
    return {'base_value' : base_value, 'a' : list(res.x)[0], 'b' : list(res.x)[1]}

In [4]:
# функция получения payment profile (т.е. по сути кривой накопления revenue)

def get_payment_profile(params, profile_range):
    [c, a, b] = params.values()
    
    if a == 0 and b == 0:
        return [1] * (profile_range)
    
    t = []
    for i in range(0, profile_range):
        t.append(model_function(c, a, b, i))
    
    return t

In [5]:
# функция подсчета ошибки построения payment profile

def get_profile_error(real_data, approximation):
    profile_error = pd.DataFrame.from_dict({
            'lt' : range(len(real_data)), 
            'real_data' : real_data, 
            'approximation' : approximation
    })
    
    profile_error['real_data'] = profile_error['real_data'].cumsum()
    profile_error['real_data'] = profile_error['real_data'] / profile_error['real_data'].max()
    
    profile_error['approximation'] = profile_error['approximation'].cumsum()
    profile_error['approximation'] = profile_error['approximation'] / profile_error['approximation'].max()
    
    profile_error['error'] = profile_error['approximation'] / profile_error['real_data'] - 1
    profile_error['error'] = (profile_error['error'] * 100).apply(round).astype(int).astype(str) + '%'

    return profile_error

In [7]:
# Для примера на входном dataframe мы построили линейку в одном случае на 1-ом дне, в другом на недельных данных

input_dataframe = pd.read_csv('input_dataframe.csv')

# to_approx = input_dataframe.loc[(input_dataframe.install_date == '2021-06-01') & (input_dataframe.platform == 'android')]['iap_revenue'].tolist()
to_approx = input_dataframe.loc[(input_dataframe.install_date >= '2021-06-01') & (input_dataframe.install_date <= '2021-06-07') & (input_dataframe.platform == 'android')].groupby('lt')['iap_revenue'].sum().tolist()

In [8]:
# ПРИМЕР 1. ВОТ В ЧЕМ ЗАДАЧА - пример апроксимации в лоб, т.е. по сути оверфитинга

# Когда мы применяем функцию построения payment profile 90 дней на самих 90 днях, 
# то ошибка выглядит "ок", от 0 до 7% в первые дни, при том что на старших днях в районе 0

coefs = get_coefs(to_approx[0:90])

p_profile = get_payment_profile(coefs, len(to_approx))

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(get_profile_error(to_approx, p_profile))

Optimization terminated successfully.
         Current function value: 237.452906
         Iterations: 66
         Function evaluations: 127


,lt,real_data,approximation,error
0,0,0.077936,0.078182,0%
1,1,0.120348,0.118313,-2%
2,2,0.148299,0.152193,3%
3,3,0.178011,0.182293,2%
4,4,0.206022,0.209694,2%
5,5,0.244030,0.235008,-4%
6,6,0.278834,0.258631,-7%
7,7,0.300366,0.280839,-7%
8,8,0.318666,0.301836,-5%
9,9,0.338771,0.321779,-5%


In [6]:
# ПРИМЕР 2. ВОТ В ЧЕМ ЗАДАЧА - а вот здесь большая проблема

# Когда мы применяем функцию построения payment profile 90 дней только на 30 днях, 
# т.к. по бизнеc необходимости мы не можем использовать только "старые" линейки прожившие 90 дней, 
# а зачастую хотим использовать последние 30 дней или иногда даже меньше, то мы строим функцию на первых n днях
# а применяем ко всей длине payment profile (на данном проекте это 90 дней, но может быть и больше и 365)

# и тут мы видим ошибку - payment profile построился на первых 30 днях и по факту стабильно начинает занижать 
# по сравнению с фактическими данными, Т.е. ошибка стабильно держится порядка 10% с первого до 30-ого дня, 
# что связано с тем что на фактических данных 90 дней после 30-ого дня было 1-59,2% = 40,8% 
# а в модельной апроксимации на 30-ти днях только  1-65,6% = 34,4%

coefs = get_coefs(to_approx[0:30])

p_profile = get_payment_profile(coefs, len(to_approx))

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(get_profile_error(to_approx, p_profile))

Optimization terminated successfully.
         Current function value: 328.220804
         Iterations: 65
         Function evaluations: 129


,lt,real_data,approximation,error
0,0,0.077936,0.086090,10%
1,1,0.120348,0.132502,10%
2,2,0.148299,0.171315,16%
3,3,0.178011,0.205473,15%
4,4,0.206022,0.236293,15%
5,5,0.244030,0.264528,8%
6,6,0.278834,0.290667,4%
7,7,0.300366,0.315056,5%
8,8,0.318666,0.337949,6%
9,9,0.338771,0.359541,6%
